# Add Traffic Signals to Network
In this case we already have a traffic signal inventory from the Georgia Department of Transportation, but code for downloading existing traffic signal data from OpenStreetMap (OSM) is also included. This code also retrieves crossings from OSM.

First, we'll find signals in the GDOT data that are not covered in the OSM data. Then we'll take these intersect the them with OSM nodes. Using the reference ID columns on the OSM links, these signals can be added to the links. A turn dataframe is then constructed with all of the links that contain at least one signal.

Once this happens, the link street name will be cross-referenced to validate the match.

- If signals are both road links, check the road name on each to see if it matches the GDOT name
- If non-road link just set it to null for now. Some of these may be crosswalks at the intersection, but they could also be walkways further away from the intersection.

This final result should be QAQC'd.

In [ ]:
from pathlib import Path 
import geopandas as gpd
import pandas as pd
import requests

In [ ]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
osm_download_fp = Path(config['project_directory']) / 'OSM_Download'
network_fp = Path(config['project_directory']) / 'Network'
if network_fp.exists() == False:
    network_fp.mkdir()

In [ ]:
links = gpd.read_file(network_fp/'networks.gpkg',layer='osm_links')
nodes = gpd.read_file(network_fp/'networks.gpkg',layer='osm_nodes')

In [ ]:
raw = gpd.read_file(osm_download_fp/f"osm_{config['geofabrik_year']}.gpkg",layer="raw",ignore_geometry=True)
links = pd.merge(links,raw[['osmid','name','oneway']],how='left',on='osmid')

In [ ]:
links['oneway'] = links['oneway'].isin(['yes','-1'])

In [ ]:
#buffer function
def buffer_signals(signal_gdf,buffer_ft):
    '''
    Use to create a copy of a gdf and buffer the point geometry
    '''
    signal_gdf = signal_gdf.copy()
    signal_gdf.geometry = signal_gdf.buffer(buffer_ft)
    return signal_gdf

# Get OSM traffic signals

In [ ]:
osm_signals = gpd.read_file(osm_download_fp/f"osm_{config['geofabrik_year']}.gpkg",layer='highway_nodes')
osm_signals = osm_signals[osm_signals['highway']=='traffic_signals']
osm_signals.to_crs(config['projected_crs_epsg'],inplace=True)
osm_signal_ids = set(osm_signals['osmid'].tolist())

In [ ]:
signalized_links = links[(links['osm_A'].isin(osm_signal_ids)) | (links['osm_B'].isin(osm_signal_ids))]
import src.modeling_turns as modeling_turns
#TODO change this to not create the turn graph (just make it an extra optional step)
## Create turn graph dataframe
_, turns_df = modeling_turns.create_pseudo_dual_graph(signalized_links,'osm_A','osm_B','osm_linkid','oneway')

#add signals ids back in
turns_df['signalized'] = turns_df['source_B'].isin(osm_signal_ids)

In [ ]:
turns_df.drop(columns=['source','target'],inplace=True)

In [ ]:
turns_df.to_parquet(network_fp/'osm_signals.parquet')

## GDOT Signals

In [ ]:
gdot_signals_fp = Path('D:/RAW/GDOT/traffic_signals.geojson')
keep = ['signalID','mainStreetName','sideStreetName','geometry']
gdot_signals = gpd.read_file(gdot_signals_fp).to_crs(links.crs)[keep]
#remove gdot_signals that are beyond the network boundaries
gdot_signals = gdot_signals.clip(links.total_bounds)
gdot_signals.head()

## Symmetric Difference
Find GDOT signals that are not already covered by OSM. Maybe ask an undergrad to add these into OSM later.

In [ ]:
print(gdot_signals.shape[0],'GDOT signals and',osm_signals.shape[0],'OSM signals')

In [ ]:
# #try the buffer in 50 ft intervals to see results
# for buffer_ft in range(50,501,50):
#     #buffer_ft = 100
#     buffered_signals = osm_signals.copy()
#     buffered_signals.geometry = buffered_signals.buffer(buffer_ft)
#     difference = gdot_signals.overlay(buffered_signals,how='difference')
#     print(buffer_ft,difference.shape[0])
buffer_ft = 100 #selecting 100 ft based on city block sizes and that's about where the number of gdot signals not represented by osm signals drops
buffered_osm_signals = buffer_signals(osm_signals,buffer_ft)
difference = gdot_signals.overlay(buffered_osm_signals,how='difference')
difference = difference.drop_duplicates()
# plot if desired
# m = difference.explore()
# osm_signals.explore(m=m,style_kwds={'color':'red'})

In [ ]:
difference.shape[0]

In [ ]:
# difference.to_file(Path.home()/'testing.gpkg')

## Add these new GDOT signals into the OSM network
Set a buffer for each signal and find all candidate nodes associated
- First we set a buffer distance around each signal to find all candidate nodes associated with the traffic signal.

In [ ]:
buffer_ft = 100
gdot_buffered = buffer_signals(difference,buffer_ft)
candidate_signals = gpd.overlay(nodes,gdot_buffered,how="intersection")

In [ ]:
#To
#candidate_signals.explore()

In [ ]:
links['link_type'].unique()

In [ ]:
#only consider road nodes
only_roads = links['link_type'].isin(['road'])
road_nodes = links['osm_A'].append(links['osm_B']).value_counts()
# and remove matches where degree is 2 or less
road_nodes = road_nodes[road_nodes>2].index.tolist()

In [ ]:
candidate_signals = candidate_signals[candidate_signals['osm_N'].isin(road_nodes)]

In [ ]:
test_dict = dict(zip(candidate_signals['osm_N'],candidate_signals['signalID']))

links['signal_A'] = links['osm_A'].map(test_dict)
links['signal_B'] = links['osm_B'].map(test_dict)

In [ ]:
candidate_signals_links = links[(links['signal_A'] != links['signal_B']) & (links['signal_A'].notna() | links['signal_B'].notna()) & (links['link_type']=='road')]

In [ ]:
#candidate_signals_links.explore()

In [ ]:
candidate_signals_links.columns

In [ ]:
import src.modeling_turns as modeling_turns
#TODO change this to not create the turn graph (just make it an extra optional step)
## Create turn graph dataframe
_, turns_df = modeling_turns.create_pseudo_dual_graph(candidate_signals_links,'osm_A','osm_B','osm_linkid','oneway')

In [ ]:
#add signals ids back in
test_dict = dict(zip(candidate_signals['osm_N'],candidate_signals['signalID']))

turns_df['source_signal_A'] = turns_df['source_A'].map(test_dict)
turns_df['source_signal_B'] = turns_df['source_B'].map(test_dict)
turns_df['target_signal_A'] = turns_df['target_A'].map(test_dict)
turns_df['target_signal_B'] = turns_df['target_B'].map(test_dict)
#assign the source signal and target signal parts based on the link directions
import numpy as np
turns_df['source_signal'] = np.where(turns_df['source_reverse_link'], turns_df['source_signal_A'], turns_df['source_signal_B'])
turns_df['target_signal'] = np.where(turns_df['target_reverse_link']==False, turns_df['source_signal_B'], turns_df['source_signal_A'])
turns_df.drop(columns=['source_signal_A','source_signal_B','target_signal_A','target_signal_B'],inplace=True)

In [ ]:
#signal ids must be the same to be considered a signalized turn
turns_df = turns_df[turns_df['source_signal'] == turns_df['target_signal']]

In [ ]:
#add name and the signal cross street names back
name_dict = dict(zip(links['osm_linkid'],links['name']))

turns_df['source_name'] = turns_df['source_linkid'].map(name_dict)
turns_df['target_name'] = turns_df['target_linkid'].map(name_dict)

In [ ]:
candidate_signals

In [ ]:
candidate_signals_links = links[(links['signal_A'] == links['signal_B']) & (links['link_type']=='road')]

In [ ]:
#add name
pd.merge(candidate_signals_links,candidate_signals.drop(columns=['geometry']),on='')

In [ ]:
A = pd.merge(links[['osm_A']],candidate_signals.drop(columns=['geometry']),left_on='osm_A',right_on='osm_N',how='left')
# A.drop(columns=['osm_A','osm_N'],inplace=True)
# A.columns = A.columns + '_A'

B = pd.merge(links[['osm_B']],candidate_signals.drop(columns=['mainStreetName','sideStreetName','geometry']),left_on='osm_B',right_on='osm_N',how='left')
# B.drop(columns=['osm_B','osm_N'],inplace=True)
# B.rename(columns={'signalID':'signalID_B'},inplace=True)


In [ ]:
links[['osm_A']]

In [ ]:
B.shape[0]

In [ ]:
A.shape[0] 

In [ ]:

test = pd.concat([links,A,B],axis=1)


In [ ]:
test

In [ ]:
candidate_signals

In [ ]:

intersect = {key:item for key, item in intersect.items() if key in road_nodes}

In [ ]:

signals['buffered_geo'] = signals.buffer(buffer_ft)
signals.set_geometry('buffered_geo',inplace=True)
#signals.explore()

## Next, we intersect these bufferred signals with the street nodes

In [ ]:
intersect = gpd.overlay(nodes,signals,how='intersection')
intersect.head()
# intersect = intersect[['N','signalID']]#,'mainStreetName','sideStreetName']]
# intersect = dict(zip(intersect['N'],intersect['signalID']))

## First identify public road intersections 
Most signals should only be at the intersection of public roads (and maybe some major parking lot/service road entrances), knowing this subset to only look at public roads and then calculate the degree of the road nodes. Remove signal id matches for links with degree of 2 or less.

In [ ]:
links['link_type'].unique()

In [ ]:
only_roads = links['link_type'].isin(['road','service'])
road_nodes = links['A'].append(links['B']).value_counts()
#remove matches where degree is 2 or less
road_nodes = road_nodes[road_nodes>2].index.tolist()
intersect = {key:item for key, item in intersect.items() if key in road_nodes}

In [ ]:
only_roads = links['link_type']=='road'
road_nodes = links['A'].append(links['B']).value_counts()
#remove matches where degree is 2 or less
road_nodes = road_nodes[road_nodes>2].index.tolist()
intersect = {key:item for key, item in intersect.items() if key in road_nodes}

### With that done, we assign the signal ID to the node and add it as an attribute in links

In [ ]:
links['signal_A'] = links['A'].map(intersect)
links['signal_B'] = links['B'].map(intersect)
nodes['signalid'] = nodes['N'].map(intersect)

Drop null values

In [ ]:
links = links[~links[['signal_A','signal_B']].isna().all(axis=1)]
nodes = nodes[~nodes['N'].isna()]

## In the Export Network notebook, we'll process this data further

In [ ]:
links.to_file(network_fp/'signals_added.gpkg',layer='links')
nodes.to_file(network_fp/'signals_added.gpkg',layer='nodes')